# Set UP

In [1]:
import os, os.path
from datetime import date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import re

from pandas.plotting import register_matplotlib_converters
import seaborn as sns
sns.set(
    font_scale=1.5,
    style="darkgrid",
    rc={'figure.figsize':(20,7)})

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.precision',3)

# Preparing the datasets

### Concatening the csv extracted from Liquipedia

There are several ways to get data from liquipedia.net/counterstrike's API. You can send requests from a Python script ([see Part 01 of this series](https://github.com/ThCord/CSGO_Team-analysis/blob/master/CSGO_Teams-Part00_fetching%20data%20through%20the%20API.ipynb)) or use their "search engine" that does the same queries. Though you can only download one 500 rows max CSV at a time.

In our case, that means we will have to concatenate 3 of them since liquipedia.net counts 1013 CS:GO teams.

Depending on when you get these data, the overall figures might evolve drastically - it's logical, since liquipedia.net updates its informations as soon as they happen.

To directly download the 3 csv I use you can follow this [link](https://liquipedia.net/counterstrike/index.php?title=Special:Ask&limit=500&offset=0&q=%5B%5BCategory%3ACSGO+Teams%5D%5D%0A&p=format%3Dbroadtable%2Flink%3Dall%2Fheaders%3Dshow%2Fsearchlabel%3D...-20further-20results%2Fclass%3Dsortable-20wikitable-20smwtable&po=%3FHas+name%0A%3FIs+active%0A%3FWas+created%0A%3FWas+disbanded%0A%3FHas+earnings%0A%3FHas+location%0A%3FHas+region%0A%3FHas+sponsor%0A%3FHas+earnings+in+2012%0A%3FHas+earnings+in+2013%0A%3FHas+earnings+in+2014%0A%3FHas+earnings+in+2015%0A%3FHas+earnings+in+2016%0A%3FHas+earnings+in+2017%0A%3FHas+earnings+in+2018%0A%3FHas+earnings+in+2019%0A%3FHas+site%0A%3FHas+twitter%0A%3FHas+twitch+stream%0A%3FHas+instagram%0A%3FHas+facebook%0A%3FHas+youtube+channel%0A%3FHas+faceit+profile%0A%3FHas+vk%0A%3FHas+esea+id%0A%3FHas+steam+profile%0A&sort=&order=asc&eq=yes#search), it will lead you to the dedicated Liquipedia Counter Strike page.

    

In [2]:
# if you did not place your datasets in the same directory as your ipynb or py file modify the next line to match their actual location
path = os.getcwd() + r"\Data"

dfs = []
for file in os.listdir(path):
    directories = path + "\\" + file #change the "\\" according to you system if you're not on Windows
    df = pd.read_csv(directories) 
    dfs.append(df)
    
CSGO_Teams = pd.concat(dfs, ignore_index=True)
duplicateRowsDF = CSGO_Teams[CSGO_Teams.duplicated(keep='first')]
nbrDuplicates = len(duplicateRowsDF)
CSGO_Teams = CSGO_Teams.drop_duplicates()
print("\nConcatening over:")

#sanity check
if nbrDuplicates != 0:
    print("There were",nbrDuplicates, "duplicates that were deleted from the dataframe.")
else:
    print("\tNo duplicated rows detected.")
print("\tNbr of rows in the dataframe :",CSGO_Teams.shape[0])
print("\tNbr columns in the dataframe :",CSGO_Teams.shape[1])

    



Concatening over:
	No duplicated rows detected.
	Nbr of rows in the dataframe : 1013
	Nbr columns in the dataframe : 26


In [3]:
CSGO_Teams.sample(3)

,Unnamed: 0,Has name,Is active,Was created,Was disbanded,Has earnings,Has location,Has region,Has sponsor,Has earnings in 2012,Has earnings in 2013,Has earnings in 2014,Has earnings in 2015,Has earnings in 2016,Has earnings in 2017,Has earnings in 2018,Has earnings in 2019,Has site,Has twitter,Has twitch stream,Has instagram,Has facebook,Has youtube channel,Has faceit profile,Has vk,Has esea id
796,Team Turkey,Team Turkey,True,2004,NaN,50254,Turkey,Europe,NaN,0,0,0,0,50000,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan
912,XPC Gaming Ladies,XPC Gaming Ladies,False,2015-10-01,2016,0,Romania,Europe,"CShub.net, HeflaTV, GamersApparel",0,0,0,0,0,0,0,0,http://xpc-gaming.com,NaN,https://www.twitch.tv/xpcgaming,NaN,https://facebook.com/XPC.eSports,https://www.youtube.com/channel/UCYLWf2PETD7xGw2LSFhpKJw,nan,NaN,nan
532,Property Female,Property Female,False,2010,7 February 2017,150,Sweden,Europe,[http://comviq.se/student COMVIQ]<br />[http://www.steelseries.com/ SteelSeries]<br />[http://ww...,0,0,0,150,0,0,0,0,http://www.teamproperty.net/,https://twitter.com/Team_Property,https://www.twitch.tv/Team_Property,NaN,https://facebook.com/TeamprOprty,https://www.youtube.com/TeamprOperty,nan,NaN,nan


In [4]:
CSGO_Teams.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1013 entries, 0 to 1012
Data columns (total 26 columns):
Unnamed: 0              1013 non-null object
Has name                1013 non-null object
Is active               1013 non-null bool
Was created             779 non-null object
Was disbanded           703 non-null object
Has earnings            1013 non-null int64
Has location            1001 non-null object
Has region              1009 non-null object
Has sponsor             525 non-null object
Has earnings in 2012    1013 non-null int64
Has earnings in 2013    1013 non-null int64
Has earnings in 2014    1013 non-null int64
Has earnings in 2015    1013 non-null int64
Has earnings in 2016    1013 non-null int64
Has earnings in 2017    1013 non-null int64
Has earnings in 2018    1013 non-null int64
Has earnings in 2019    1013 non-null int64
Has site                578 non-null object
Has twitter             638 non-null object
Has twitch stream       222 non-null object
Has instag

### Adding other statistical informations

We'll add some other informations about the medals each team has won. Since this didn't show up with the API we will use a Pandas feature that is great for parsing html tables : pd.read_html()

In [5]:
Stats = pd.read_html("https://liquipedia.net/counterstrike/Statistics/Total")
dfs_stats = []
for i in range(len(Stats)):
    df = Stats[i]
    dfs_stats.append(df)

df_medals = pd.concat(dfs_stats, ignore_index=True)
df_medals.drop_duplicates(inplace=True) 
df_medals.drop("Earnings",axis=1,inplace=True)
df_medals.rename(columns={'Organization':'Has name',
                                     'Unnamed: 1':'Gold_medals',
                                     'Unnamed: 2':'Silver_medals',
                                     'Unnamed: 3':'Bronze_medals'},
                                     inplace=True)       

df_medals["Total_Medals"] = df_medals.Gold_medals + df_medals.Silver_medals + df_medals.Bronze_medals


In [6]:
df_medals.shape

(600, 5)

In [7]:
df_medals.sort_values("Total_Medals",ascending=False).head()

,Has name,Gold_medals,Silver_medals,Bronze_medals,Total_Medals
1,Fnatic,85,63,25,173
6,Virtus.pro,60,37,22,119
3,SK Gaming,58,36,16,110
7,mousesports,60,28,20,108
8,Ninjas in Pyjamas,62,30,12,104


### Checking for potential multiple entries

In [8]:
CSGO_Teams[CSGO_Teams["Has name"].duplicated()]
df_medals[df_medals["Has name"].duplicated()]

,Unnamed: 0,Has name,Is active,Was created,Was disbanded,Has earnings,Has location,Has region,Has sponsor,Has earnings in 2012,Has earnings in 2013,Has earnings in 2014,Has earnings in 2015,Has earnings in 2016,Has earnings in 2017,Has earnings in 2018,Has earnings in 2019,Has site,Has twitter,Has twitch stream,Has instagram,Has facebook,Has youtube channel,Has faceit profile,Has vk,Has esea id
812,Team X (Danish team),Team X,False,2016-06-30,26 August 2016,50000,Denmark,Europe,NaN,0,0,0,0,50000,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan


,Has name,Gold_medals,Silver_medals,Bronze_medals,Total_Medals
55,MIBR,27,10,6,43
67,Counter Logic Gaming,6,5,0,11
498,OverGaming,1,4,0,5


Only three entries are duplicated in these datasets : 
    - 1 in CSGO_Teams
    - 3 in df_medals
    
This needs to be investigated since it will biais our analysis after our incoming merge.

Before droping these rows by their indexes let's take a look at them.

In [9]:
CSGO_Teams[CSGO_Teams["Has name"].str.contains("Team X")]

,Unnamed: 0,Has name,Is active,Was created,Was disbanded,Has earnings,Has location,Has region,Has sponsor,Has earnings in 2012,Has earnings in 2013,Has earnings in 2014,Has earnings in 2015,Has earnings in 2016,Has earnings in 2017,Has earnings in 2018,Has earnings in 2019,Has site,Has twitter,Has twitch stream,Has instagram,Has facebook,Has youtube channel,Has faceit profile,Has vk,Has esea id
811,Team X (Swedish team),Team X,False,2011,13 June 2013,0,Sweden,Europe,NaN,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan
812,Team X (Danish team),Team X,False,2016-06-30,26 August 2016,50000,Denmark,Europe,NaN,0,0,0,0,50000,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan
813,Team XENEX,Team XENEX,False,[[File:Csgo icon.png|link=Counter-Strike: Global Offensive]] 2014-08-31,3 September 2018,9969,United Kingdom,Europe,[http://www.gamersapparel.co.uk Gamers Apparel]<br>[http://www.creative.com Creative],0,0,0,6141,188,1173,2468,0,NaN,https://twitter.com/TeamXENEX,NaN,NaN,https://facebook.com/TeamXENEX,NaN,nan,NaN,nan


Actually there were two different teams that used the same name "Team X", one was from Sweeden and the other one from Danemark. We'll need to specify that in the cells.

In [10]:
CSGO_Teams.loc[811, 'Has name'] = "Team X (Swedish team)"
CSGO_Teams.loc[812, 'Has name'] = "Team X (Danish team)"

Now let's take a look at the duplicate entries in df_medals :

In [11]:
df_medals[df_medals["Has name"].str.contains("Counter Logic Gaming")]

df_medals[df_medals["Has name"] == "MIBR"]

df_medals[df_medals["Has name"] == "OverGaming"]

,Has name,Gold_medals,Silver_medals,Bronze_medals,Total_Medals
40,Counter Logic Gaming,8,3,3,14
67,Counter Logic Gaming,6,5,0,11


,Has name,Gold_medals,Silver_medals,Bronze_medals,Total_Medals
18,MIBR,5,2,3,10
55,MIBR,27,10,6,43


,Has name,Gold_medals,Silver_medals,Bronze_medals,Total_Medals
314,OverGaming,0,0,0,0
498,OverGaming,1,4,0,5


The double entries in df_medals are a bit more challenging to handle. According to Liquiepedia there are also two teams that share the name "Counter Logic Gaming". They are from the same organization, but the one still active has a "Red" addendum to its name that is missing in the dataset - hence the double entry...

Therefore, according to Liquiepedia, at index 40 is "Counter Logic Gaming" and the one at index 67 is "Counter Logic Gaming Red". 

In [12]:
df_medals.loc[67, 'Has name'] = "Counter Logic Gaming Red"

Regarding MIBR team, there are actually two different entries on Liquiepdia : one where all letters are capital wich is a CS:GO team and the other where just first letter is in capital, who was focused in Counter-Strike and Counter-Strike Source. 

Since our study is about CS:GO and because the "Mibr" entry is, therefore, not in our CSGO_Teams dataset we will get rid of this one in the df_medals dataset (index 54). 



In [13]:
df_medals.drop([55],axis=0,inplace=True)

df_medals[df_medals["Has name"] == "MIBR"]

,Has name,Gold_medals,Silver_medals,Bronze_medals,Total_Medals
18,MIBR,5,2,3,10


For OverGaming, since the first row has no medal, we will simple delete it.

In [14]:
df_medals.drop([314],axis=0,inplace=True)

df_medals[df_medals["Has name"].str.contains("OverGaming")]

,Has name,Gold_medals,Silver_medals,Bronze_medals,Total_Medals
498,OverGaming,1,4,0,5


### Merging the two dataframes 

Now it's time to combine these two dataframes with a left join and our dataset will be ready for some cleaning.

In [15]:
CSGO_Merged = pd.merge(CSGO_Teams,df_medals,how="left",on="Has name",indicator=True)

CSGO_Merged.head()

,Unnamed: 0,Has name,Is active,Was created,Was disbanded,Has earnings,Has location,Has region,Has sponsor,Has earnings in 2012,Has earnings in 2013,Has earnings in 2014,Has earnings in 2015,Has earnings in 2016,Has earnings in 2017,Has earnings in 2018,Has earnings in 2019,Has site,Has twitter,Has twitch stream,Has instagram,Has facebook,Has youtube channel,Has faceit profile,Has vk,Has esea id,Gold_medals,Silver_medals,Bronze_medals,Total_Medals,_merge
0,/10/,/10/,False,2013,27 April 2014,3900,Germany,Europe,NaN,0,3900,0,0,0,0,0,0,NaN,NaN,NaN,NaN,https://facebook.com/team10cs,NaN,nan,NaN,nan,nan,nan,nan,nan,left_only
1,100 Thieves,100 Thieves,True,2017-11-20 <br>[[File:Csgo icon.png|link=Counter-Strike: Global Offensive]]: 2017-12-12,NaN,50000,United States,North America,[https://www.rocketmortgage.com/ Rocket mortgage]<br />[https://www.redbull.com/us-en/ Red Bull]...,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both
2,1337,1337,False,NaN,15 July 2015,6650,Sweden,Europe,NaN,0,0,150,6500,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,0.000,2.000,0.000,2.000,both
3,2Kill Gaming,2Kill Gaming,False,[[File:Csgo icon.png|link=Counter-Strike: Global Offensive]] : 2013-05-17,2018,6497,Brazil,South America,[http://www.kappabrasil.com.br Kappa]<br />[http://arsenalxfire.com.br Arsenal XFire]<br />[http...,0,0,0,0,3255,3242,0,0,https://www.2killgaming.com,https://twitter.com/2KillGaming,NaN,https://www.instagram.com/2killgaming,https://facebook.com/2killgaming,https://www.youtube.com/channel/2KillGaming,nan,NaN,nan,3.000,3.000,1.000,7.000,both
4,31337 eSPORTS,31337 eSPORTS,False,2001-02-05<br>\n[[File:Csgo icon.png|link=Counter-Strike: Global Offensive]] 2012-01-01,1 February 2018,1052,Poland,Europe,NaN,0,0,0,792,260,0,0,0,http://eleet-esports.pl/,https://twitter.com/31337GAMING,NaN,NaN,https://facebook.com/31337esportspl,NaN,nan,NaN,nan,nan,nan,nan,nan,left_only


# Cleaning

To do :

- Changing column names
- Remove the "unnamed: 0" column
- Changing "Is active" results from booleans to Active/Inactive
- Cleaning the "Was created" and "Was disbanded" to only keep the year
- Extracting the sponsors names in the "Has sponsor" column 
- Rearanging columns order

### Changing columns name

In [16]:
CSGO_Merged.columns

Index(['Unnamed: 0', 'Has name', 'Is active', 'Was created', 'Was disbanded', 'Has earnings', 'Has location', 'Has region', 'Has sponsor', 'Has earnings in 2012', 'Has earnings in 2013', 'Has earnings in 2014', 'Has earnings in 2015', 'Has earnings in 2016', 'Has earnings in 2017', 'Has earnings in 2018', 'Has earnings in 2019', 'Has site', 'Has twitter', 'Has twitch stream', 'Has instagram', 'Has facebook', 'Has youtube channel', 'Has faceit profile', 'Has vk', 'Has esea id', 'Gold_medals', 'Silver_medals', 'Bronze_medals', 'Total_Medals', '_merge'], dtype='object')

In [17]:
CSGO_Merged.rename(columns={'Has name': "team", 
                            'Is active':"status",
                            'Was created':"creation_date",
                            'Was disbanded':"disbanding_date",
                            'Has earnings':"total_gains",
                            'Has location': "country",
                            'Has region': "region",
                            'Has sponsor':"sponsors",
                            'Has earnings in 2012': "2012_gains",
                            'Has earnings in 2013': "2013_gains",
                            'Has earnings in 2014': "2014_gains",
                            'Has earnings in 2015': "2015_gains",
                            'Has earnings in 2016': "2016_gains",
                            'Has earnings in 2017': "2017_gains",
                            'Has earnings in 2018': "2018_gains",
                            'Has earnings in 2019': "2019_gains",
                            'Has site' : "website",
                            'Has twitter': "twitter_account",
                            'Has twitch stream': "twitch_account",
                            'Has instagram': "instagram_account",
                            'Has facebook' : "facebook_account",
                            'Has youtube channel': "youtube_channel",
                            'Has faceit profile': "faceit_profile",
                            'Has vk': "VKontakte_account",
                            'Has esea id': "esea_id",
                            'Gold_medals':"gold_medals",
                            'Silver_medals': "silver_medals",
                            'Bronze_medals': "bronze_medals",
                            'Total_Medals':"total_medals"},
                                     inplace=True)  

### Droping unnecessary columns

In [18]:
CSGO_Merged.drop("Unnamed: 0",axis=1,inplace=True)

In [19]:
CSGO_Merged.sample()

,team,status,creation_date,disbanding_date,total_gains,country,region,sponsors,2012_gains,2013_gains,2014_gains,2015_gains,2016_gains,2017_gains,2018_gains,2019_gains,website,twitter_account,twitch_account,instagram_account,facebook_account,youtube_channel,faceit_profile,VKontakte_account,esea_id,gold_medals,silver_medals,bronze_medals,total_medals,_merge
235,Evil Geniuses,True,1999-??-??,NaN,555587,United States,North America,[http://www.monsterenergy.com/ Monster Energy]<br />[http://www.xfinity.com/gaming xfinity]<br /...,0,0,0,0,0,0,0,390500,http://www.evilgeniuses.gg/,https://twitter.com/EvilGeniuses,https://www.twitch.tv/team/eg,https://www.instagram.com/evilgeniuses,https://facebook.com/EvilGeniuses,https://www.youtube.com/myEGnet,nan,https://vk.com/evilgeniuses,nan,14.000,4.000,8.000,26.000,both


### Changing "Is active" results from booleans to Active/Inactive

The former "Is active" column is filled with boolean objects. We want to transform them into "Active" and "Inactive".

In [20]:
CSGO_Merged.status.value_counts(dropna=False)

False    703
True     310
Name: status, dtype: int64

In [21]:
CSGO_Merged.status = CSGO_Merged.status.map({False: 'Inactive', True: 'Active'})
CSGO_Merged.sample(2)

,team,status,creation_date,disbanding_date,total_gains,country,region,sponsors,2012_gains,2013_gains,2014_gains,2015_gains,2016_gains,2017_gains,2018_gains,2019_gains,website,twitter_account,twitch_account,instagram_account,facebook_account,youtube_channel,faceit_profile,VKontakte_account,esea_id,gold_medals,silver_medals,bronze_medals,total_medals,_merge
358,Japaleno,Active,2016-08-??,NaN,21203,Sweden,Europe,NaN,0,0,0,0,720,14983,3000,2500,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,14.000,4.000,0.000,18.000,both
761,Team Plantronics,Inactive,NaN,2017,5102,USA,Europe,NaN,0,0,0,0,3402,1700,0,0,https://www.plantronics.com/us/gaming/,https://twitter.com/PLTgaming,NaN,NaN,https://facebook.com/PlantronicsGamingNOR,NaN,nan,NaN,nan,nan,nan,nan,nan,left_only


###  Cleaning the "Was created" and "Was disbanded" to only keep the year

First, let's find out how many empty values are in these columns.

In [22]:
CSGO_Merged.creation_date.isnull().sum()
CSGO_Merged.creation_date.isnull().mean()*100

234

23.099703849950643

In [23]:
CSGO_Merged.disbanding_date.isnull().sum()
CSGO_Merged.disbanding_date.isnull().mean()*100

310

30.60217176702863

If the 310 null values in the "disbanding_date" columns correspond to the 310 teams that are still active, the 234 null values in the "creation_date"column are simply missing : we have no data regarding them. That's 23% of the total dataframe.

In addition, there are three rows (index 466, 605 and 720) not considered "null" where the creation dates is unclear with patterns like "20??-??" or "????-??-??". They will be turned into NaN once we will use regular expressions to extract our points of interest :
    - the year of creation of the team
    - the year the team started playing or switched to CS:GO 
    - the year of their disbanding - if any
    
Then all these columns will be converted into datetime objects.

In [24]:
Switch2CSGO = CSGO_Merged.creation_date.str.extract(".*(\d{4}).*$")
CSGO_Merged.creation_date = CSGO_Merged.creation_date.str.extract("(\d{4})")
CSGO_Merged.disbanding_date = CSGO_Merged.disbanding_date.str.extract("(\d{4})")

CSGO_Merged.insert(3,"starts_playing_CSGO", Switch2CSGO)

In [25]:
CSGO_Merged.creation_date =  pd.to_datetime(CSGO_Merged.creation_date)
CSGO_Merged.creation_date = CSGO_Merged.creation_date.dt.to_period('Y')

CSGO_Merged.disbanding_date =  pd.to_datetime(CSGO_Merged.disbanding_date)
CSGO_Merged.disbanding_date = CSGO_Merged.disbanding_date.dt.to_period('Y')

CSGO_Merged.starts_playing_CSGO =  pd.to_datetime(CSGO_Merged.starts_playing_CSGO,errors="coerce")
CSGO_Merged.starts_playing_CSGO = CSGO_Merged.starts_playing_CSGO.dt.to_period('Y')


In [26]:
CSGO_Merged.head()

,team,status,creation_date,starts_playing_CSGO,disbanding_date,total_gains,country,region,sponsors,2012_gains,2013_gains,2014_gains,2015_gains,2016_gains,2017_gains,2018_gains,2019_gains,website,twitter_account,twitch_account,instagram_account,facebook_account,youtube_channel,faceit_profile,VKontakte_account,esea_id,gold_medals,silver_medals,bronze_medals,total_medals,_merge
0,/10/,Inactive,2013,2013,2014,3900,Germany,Europe,NaN,0,3900,0,0,0,0,0,0,NaN,NaN,NaN,NaN,https://facebook.com/team10cs,NaN,nan,NaN,nan,nan,nan,nan,nan,left_only
1,100 Thieves,Active,2017,2017,NaT,50000,United States,North America,[https://www.rocketmortgage.com/ Rocket mortgage]<br />[https://www.redbull.com/us-en/ Red Bull]...,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both
2,1337,Inactive,NaT,NaT,2015,6650,Sweden,Europe,NaN,0,0,150,6500,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,0.000,2.000,0.000,2.000,both
3,2Kill Gaming,Inactive,2013,2013,2018,6497,Brazil,South America,[http://www.kappabrasil.com.br Kappa]<br />[http://arsenalxfire.com.br Arsenal XFire]<br />[http...,0,0,0,0,3255,3242,0,0,https://www.2killgaming.com,https://twitter.com/2KillGaming,NaN,https://www.instagram.com/2killgaming,https://facebook.com/2killgaming,https://www.youtube.com/channel/2KillGaming,nan,NaN,nan,3.000,3.000,1.000,7.000,both
4,31337 eSPORTS,Inactive,2001,2012,2018,1052,Poland,Europe,NaN,0,0,0,792,260,0,0,0,http://eleet-esports.pl/,https://twitter.com/31337GAMING,NaN,NaN,https://facebook.com/31337esportspl,NaN,nan,NaN,nan,nan,nan,nan,nan,left_only


### Extracting the sponsors names in the "Has sponsor" column 

The data in sponsor column is a far more complex challenge than what we had so far. Not only there are several brands per rows, they are "encrypted" in html like format which make a basic cleaning or extraction with str.split(expand=True) impossible if we want to be exhaustive.

Hopefully there is nothing regex and pd.str.extractall() can solve.

First let's examine some rows.

In [27]:
CSGO_Merged.sponsors.loc[[1,3,5,22,85,720]].to_frame()

,sponsors
1,[https://www.rocketmortgage.com/ Rocket mortgage]<br />[https://www.redbull.com/us-en/ Red Bull]...
3,[http://www.kappabrasil.com.br Kappa]<br />[http://arsenalxfire.com.br Arsenal XFire]<br />[http...
5,[http://www.newhomepc.net/ NewHomePc]<br />[http://www.agroponiente.es/ Agroponiente]<br />[http...
22,NaN
85,[https://twitter.com/MuchaProLeague Mucha Pro League]<br>[https://www.ihosts3.com/ iHosts3]
720,[https://steelseries.com/ SteelSeries]<br>[http://www.tees4gamers.com/ TEES4GAMERS]


For each sponsor's name there's an url which is an interesting piece of information that we want to keep. But if a team has more than one sponsor they are all in one row, which is problematic. 

Moreover, if the main pattern is [URL1 brand1] br tag [URL2 brand2] and so on and so forth up to 18 times for a japonses team, with a lot of noticeable variations.

The presence of NaNs - where teams have no sponsors - will also hinder our sponsors extraction since pd.str.extractall(), unlike pd.str.extract(), doesn't return them. 

To get rid of that we need to transform our CSGO_Merged into a multi-index dataframe in order to be able to merge to what pd.str.extractall() will return once we'll pass some regex patterns as arguments.



In [28]:
CSGO_MI = CSGO_Merged.copy()

CSGO_MI.set_index(['team', 'country'], inplace=True)

CSGO_MI.head()


,,status,creation_date,starts_playing_CSGO,disbanding_date,total_gains,region,sponsors,2012_gains,2013_gains,2014_gains,2015_gains,2016_gains,2017_gains,2018_gains,2019_gains,website,twitter_account,twitch_account,instagram_account,facebook_account,youtube_channel,faceit_profile,VKontakte_account,esea_id,gold_medals,silver_medals,bronze_medals,total_medals,_merge
team,country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
/10/,Germany,Inactive,2013,2013,2014,3900,Europe,NaN,0,3900,0,0,0,0,0,0,NaN,NaN,NaN,NaN,https://facebook.com/team10cs,NaN,nan,NaN,nan,nan,nan,nan,nan,left_only
100 Thieves,United States,Active,2017,2017,NaT,50000,North America,[https://www.rocketmortgage.com/ Rocket mortgage]<br />[https://www.redbull.com/us-en/ Red Bull]...,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both
1337,Sweden,Inactive,NaT,NaT,2015,6650,Europe,NaN,0,0,150,6500,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,0.000,2.000,0.000,2.000,both
2Kill Gaming,Brazil,Inactive,2013,2013,2018,6497,South America,[http://www.kappabrasil.com.br Kappa]<br />[http://arsenalxfire.com.br Arsenal XFire]<br />[http...,0,0,0,0,3255,3242,0,0,https://www.2killgaming.com,https://twitter.com/2KillGaming,NaN,https://www.instagram.com/2killgaming,https://facebook.com/2killgaming,https://www.youtube.com/channel/2KillGaming,nan,NaN,nan,3.000,3.000,1.000,7.000,both
31337 eSPORTS,Poland,Inactive,2001,2012,2018,1052,Europe,NaN,0,0,0,792,260,0,0,0,http://eleet-esports.pl/,https://twitter.com/31337GAMING,NaN,NaN,https://facebook.com/31337esportspl,NaN,nan,NaN,nan,nan,nan,nan,nan,left_only


Giving names to your regex capture groups allows pd.str.extractall() to use them as names for the columns it returns.
To name a capture group put some word inside (?P<...>).
    
The pattern here is long, but not as complicated as it might look.     

In [29]:
pattern = r"(?P<url>[\w\./-]+)(?P<ToBdroped1>/?\s)(?P<brand>\b.*?)\](?P<ToBdroped2><br\s/>\[)?"

Extract_df = CSGO_MI["sponsors"].str.extractall(pattern)
Extract_df.info()
# Extract_df.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2211 entries, (100 Thieves, United States, 0) to (x6tence White, Spain, 5)
Data columns (total 4 columns):
url           2211 non-null object
ToBdroped1    2211 non-null object
brand         2211 non-null object
ToBdroped2    442 non-null object
dtypes: object(4)
memory usage: 82.5+ KB


From just this extract we can already assume that 2211 sponsor partnerships have been made between CS:GO teams and brands.

In [30]:
Extract_df.head(10)

url ToBdroped1            brand ToBdroped2
team          country       match                                                                  
100 Thieves   United States 0      //www.rocketmortgage.com/             Rocket mortgage    <br />[
                            1       //www.redbull.com/us-en/                    Red Bull    <br />[
                            2                  //stockx.com/                      stockX    <br />[
                            3             //www.totinos.com/                     Totinos    <br />[
                            4                    //cash.app/                     cashapp        NaN
2Kill Gaming  Brazil        0       //www.kappabrasil.com.br                       Kappa    <br />[
                            1          //arsenalxfire.com.br               Arsenal XFire    <br />[
                            2              //www.asrock.com/                      ASRock    <br />[
                            3       //www.seagate.com/br/pt/                     Seagate        NaN
34united Club Spain         0           //www.newhomepc.net/                   NewHomePc    <br />[

Now we go back to a single index in order to merge later.

In [31]:
Extract_df.reset_index(inplace=True)
Extract_df.drop(["match","ToBdroped1","ToBdroped2"],axis=1,inplace=True)
Extract_df.head(15)

,team,country,url,brand
0,100 Thieves,United States,//www.rocketmortgage.com/,Rocket mortgage
1,100 Thieves,United States,//www.redbull.com/us-en/,Red Bull
2,100 Thieves,United States,//stockx.com/,stockX
3,100 Thieves,United States,//www.totinos.com/,Totinos
4,100 Thieves,United States,//cash.app/,cashapp
5,2Kill Gaming,Brazil,//www.kappabrasil.com.br,Kappa
6,2Kill Gaming,Brazil,//arsenalxfire.com.br,Arsenal XFire
7,2Kill Gaming,Brazil,//www.asrock.com/,ASRock
8,2Kill Gaming,Brazil,//www.seagate.com/br/pt/,Seagate
9,34united Club,Spain,//www.newhomepc.net/,NewHomePc


In [32]:
CSGO_MI = pd.merge(CSGO_MI,Extract_df,how="left",on=["team","country"]).drop("sponsors",axis=1)
CSGO_MI.head()


,team,country,status,creation_date,starts_playing_CSGO,disbanding_date,total_gains,region,2012_gains,2013_gains,2014_gains,2015_gains,2016_gains,2017_gains,2018_gains,2019_gains,website,twitter_account,twitch_account,instagram_account,facebook_account,youtube_channel,faceit_profile,VKontakte_account,esea_id,gold_medals,silver_medals,bronze_medals,total_medals,_merge,url,brand
0,/10/,Germany,Inactive,2013,2013,2014,3900,Europe,0,3900,0,0,0,0,0,0,NaN,NaN,NaN,NaN,https://facebook.com/team10cs,NaN,nan,NaN,nan,nan,nan,nan,nan,left_only,NaN,NaN
1,100 Thieves,United States,Active,2017,2017,NaT,50000,North America,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both,//www.rocketmortgage.com/,Rocket mortgage
2,100 Thieves,United States,Active,2017,2017,NaT,50000,North America,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both,//www.redbull.com/us-en/,Red Bull
3,100 Thieves,United States,Active,2017,2017,NaT,50000,North America,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both,//stockx.com/,stockX
4,100 Thieves,United States,Active,2017,2017,NaT,50000,North America,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both,//www.totinos.com/,Totinos


Now we can find how much sponsors each team have.

In [33]:
Nbr_Sponsor = pd.crosstab(CSGO_MI["team"],CSGO_MI["brand"],margins=True).iloc[:,-1].to_frame()

# Nbr_Sponsor = Nbr_Sponsor.rename({"All":"nbr_sponsors"},axis=1)
Nbr_Sponsor = Nbr_Sponsor.rename({"All":"nbr_sponsors"},axis=1).reset_index()

# Nbr_Sponsor = Nbr_Sponsor.reset_index()
Nbr_Sponsor.sample(7)

,team,nbr_sponsors
214,Lemondogs,4
358,Team AGG,6
156,Furious Gaming,3
50,Berzerk,3
149,FlipSid3 Tactics,4
443,VeryGames,2
174,Giants Gaming,10


In [34]:
CSGO_MI = pd.merge(CSGO_MI,Nbr_Sponsor,how="left",on="team")
CSGO_MI.nbr_sponsors = CSGO_MI.nbr_sponsors.fillna(0).astype('int64') 
CSGO_MI.head(15)

,team,country,status,creation_date,starts_playing_CSGO,disbanding_date,total_gains,region,2012_gains,2013_gains,2014_gains,2015_gains,2016_gains,2017_gains,2018_gains,2019_gains,website,twitter_account,twitch_account,instagram_account,facebook_account,youtube_channel,faceit_profile,VKontakte_account,esea_id,gold_medals,silver_medals,bronze_medals,total_medals,_merge,url,brand,nbr_sponsors
0,/10/,Germany,Inactive,2013,2013,2014,3900,Europe,0,3900,0,0,0,0,0,0,NaN,NaN,NaN,NaN,https://facebook.com/team10cs,NaN,nan,NaN,nan,nan,nan,nan,nan,left_only,NaN,NaN,0
1,100 Thieves,United States,Active,2017,2017,NaT,50000,North America,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both,//www.rocketmortgage.com/,Rocket mortgage,5
2,100 Thieves,United States,Active,2017,2017,NaT,50000,North America,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both,//www.redbull.com/us-en/,Red Bull,5
3,100 Thieves,United States,Active,2017,2017,NaT,50000,North America,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both,//stockx.com/,stockX,5
4,100 Thieves,United States,Active,2017,2017,NaT,50000,North America,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both,//www.totinos.com/,Totinos,5
5,100 Thieves,United States,Active,2017,2017,NaT,50000,North America,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both,//cash.app/,cashapp,5
6,1337,Sweden,Inactive,NaT,NaT,2015,6650,Europe,0,0,150,6500,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,0.000,2.000,0.000,2.000,both,NaN,NaN,0
7,2Kill Gaming,Brazil,Inactive,2013,2013,2018,6497,South America,0,0,0,0,3255,3242,0,0,https://www.2killgaming.com,https://twitter.com/2KillGaming,NaN,https://www.instagram.com/2killgaming,https://facebook.com/2killgaming,https://www.youtube.com/channel/2KillGaming,nan,NaN,nan,3.000,3.000,1.000,7.000,both,//www.kappabrasil.com.br,Kappa,4
8,2Kill Gaming,Brazil,Inactive,2013,2013,2018,6497,South America,0,0,0,0,3255,3242,0,0,https://www.2killgaming.com,https://twitter.com/2KillGaming,NaN,https://www.instagram.com/2killgaming,https://facebook.com/2killgaming,https://www.youtube.com/channel/2KillGaming,nan,NaN,nan,3.000,3.000,1.000,7.000,both,//arsenalxfire.com.br,Arsenal XFire,4
9,2Kill Gaming,Brazil,Inactive,2013,2013,2018,6497,South America,0,0,0,0,3255,3242,0,0,https://www.2killgaming.com,https://twitter.com/2KillGaming,NaN,https://www.instagram.com/2killgaming,https://facebook.com/2killgaming,https://www.youtube.com/channel/2KillGaming,nan,NaN,nan,3.000,3.000,1.000,7.000,both,//www.asrock.com/,ASRock,4


Since the CSGO_MI dataframe is a bit big with all that brand/sponsors information - that goes as up as 18 sponsors for one japonse team - we'll add the Nbr_Sponsor dataframe to the CSGO_Merged dataframe.

So that we will have the option : whether we want only the number of sponsors for each team, or that + the name of all the sponsors.

In [35]:
CSGO_Merged.head()

,team,status,creation_date,starts_playing_CSGO,disbanding_date,total_gains,country,region,sponsors,2012_gains,2013_gains,2014_gains,2015_gains,2016_gains,2017_gains,2018_gains,2019_gains,website,twitter_account,twitch_account,instagram_account,facebook_account,youtube_channel,faceit_profile,VKontakte_account,esea_id,gold_medals,silver_medals,bronze_medals,total_medals,_merge
0,/10/,Inactive,2013,2013,2014,3900,Germany,Europe,NaN,0,3900,0,0,0,0,0,0,NaN,NaN,NaN,NaN,https://facebook.com/team10cs,NaN,nan,NaN,nan,nan,nan,nan,nan,left_only
1,100 Thieves,Active,2017,2017,NaT,50000,United States,North America,[https://www.rocketmortgage.com/ Rocket mortgage]<br />[https://www.redbull.com/us-en/ Red Bull]...,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both
2,1337,Inactive,NaT,NaT,2015,6650,Sweden,Europe,NaN,0,0,150,6500,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,0.000,2.000,0.000,2.000,both
3,2Kill Gaming,Inactive,2013,2013,2018,6497,Brazil,South America,[http://www.kappabrasil.com.br Kappa]<br />[http://arsenalxfire.com.br Arsenal XFire]<br />[http...,0,0,0,0,3255,3242,0,0,https://www.2killgaming.com,https://twitter.com/2KillGaming,NaN,https://www.instagram.com/2killgaming,https://facebook.com/2killgaming,https://www.youtube.com/channel/2KillGaming,nan,NaN,nan,3.000,3.000,1.000,7.000,both
4,31337 eSPORTS,Inactive,2001,2012,2018,1052,Poland,Europe,NaN,0,0,0,792,260,0,0,0,http://eleet-esports.pl/,https://twitter.com/31337GAMING,NaN,NaN,https://facebook.com/31337esportspl,NaN,nan,NaN,nan,nan,nan,nan,nan,left_only


In [36]:
CSGO_Merged = pd.merge(CSGO_Merged,Nbr_Sponsor,how="left",on="team")
CSGO_Merged.nbr_sponsors = CSGO_Merged.nbr_sponsors.fillna(0).astype('int64')

Col_List = ["nbr_sponsors","country","total_medals"]    
Other_cols = [i for i in CSGO_Merged if i not in Col_List]
Start = Other_cols[0:7]
End = Other_cols[7:]
CSGO_Merged = CSGO_Merged[Start + Col_List + End]


In [37]:
CSGO_Merged.head(7)

,team,status,creation_date,starts_playing_CSGO,disbanding_date,total_gains,region,nbr_sponsors,country,total_medals,sponsors,2012_gains,2013_gains,2014_gains,2015_gains,2016_gains,2017_gains,2018_gains,2019_gains,website,twitter_account,twitch_account,instagram_account,facebook_account,youtube_channel,faceit_profile,VKontakte_account,esea_id,gold_medals,silver_medals,bronze_medals,_merge
0,/10/,Inactive,2013,2013,2014,3900,Europe,0,Germany,nan,NaN,0,3900,0,0,0,0,0,0,NaN,NaN,NaN,NaN,https://facebook.com/team10cs,NaN,nan,NaN,nan,nan,nan,nan,left_only
1,100 Thieves,Active,2017,2017,NaT,50000,North America,5,United States,1.000,[https://www.rocketmortgage.com/ Rocket mortgage]<br />[https://www.redbull.com/us-en/ Red Bull]...,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,both
2,1337,Inactive,NaT,NaT,2015,6650,Europe,0,Sweden,2.000,NaN,0,0,150,6500,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,0.000,2.000,0.000,both
3,2Kill Gaming,Inactive,2013,2013,2018,6497,South America,4,Brazil,7.000,[http://www.kappabrasil.com.br Kappa]<br />[http://arsenalxfire.com.br Arsenal XFire]<br />[http...,0,0,0,0,3255,3242,0,0,https://www.2killgaming.com,https://twitter.com/2KillGaming,NaN,https://www.instagram.com/2killgaming,https://facebook.com/2killgaming,https://www.youtube.com/channel/2KillGaming,nan,NaN,nan,3.000,3.000,1.000,both
4,31337 eSPORTS,Inactive,2001,2012,2018,1052,Europe,0,Poland,nan,NaN,0,0,0,792,260,0,0,0,http://eleet-esports.pl/,https://twitter.com/31337GAMING,NaN,NaN,https://facebook.com/31337esportspl,NaN,nan,NaN,nan,nan,nan,nan,left_only
5,34united Club,Inactive,NaT,NaT,2014,1827,Europe,4,Spain,nan,[http://www.newhomepc.net/ NewHomePc]<br />[http://www.agroponiente.es/ Agroponiente]<br />[http...,1076,260,491,0,0,0,0,0,http://www.34united.es/,https://twitter.com/34united,https://www.twitch.tv/34unitedTV,NaN,https://facebook.com/34united,https://www.youtube.com/34unitedesportsclub,nan,NaN,nan,nan,nan,nan,left_only
6,3DMAX,Inactive,2009,2009,2019,86562,Europe,3,France,15.000,[https://gamersapparel.co.uk/ Gamers Apparel]<br>[https://www.avast.com/ Avast]<br>[https://fr.a...,2125,2969,7952,6538,0,0,45576,15375,https://www.3dmax.team/,https://twitter.com/3DMAXGaming,NaN,NaN,https://facebook.com/3DMAXGaming,NaN,nan,NaN,nan,9.000,4.000,2.000,both


### Rearanging columns order

First we'll create a list of the column(s) we want to move.
Then we'll create another list with the rest of the columns.
And we'll specify where we want to place or columns.

In [38]:
Col_List = ["nbr_sponsors","brand","url"]    
Other_cols = [i for i in CSGO_MI if i not in Col_List]
Start = Other_cols[0:7]
End = Other_cols[7:]
CSGO_MI = CSGO_MI[Start + Col_List + End]

In [39]:
CSGO_MI.head(15)

,team,country,status,creation_date,starts_playing_CSGO,disbanding_date,total_gains,nbr_sponsors,brand,url,region,2012_gains,2013_gains,2014_gains,2015_gains,2016_gains,2017_gains,2018_gains,2019_gains,website,twitter_account,twitch_account,instagram_account,facebook_account,youtube_channel,faceit_profile,VKontakte_account,esea_id,gold_medals,silver_medals,bronze_medals,total_medals,_merge
0,/10/,Germany,Inactive,2013,2013,2014,3900,0,NaN,NaN,Europe,0,3900,0,0,0,0,0,0,NaN,NaN,NaN,NaN,https://facebook.com/team10cs,NaN,nan,NaN,nan,nan,nan,nan,nan,left_only
1,100 Thieves,United States,Active,2017,2017,NaT,50000,5,Rocket mortgage,//www.rocketmortgage.com/,North America,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both
2,100 Thieves,United States,Active,2017,2017,NaT,50000,5,Red Bull,//www.redbull.com/us-en/,North America,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both
3,100 Thieves,United States,Active,2017,2017,NaT,50000,5,stockX,//stockx.com/,North America,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both
4,100 Thieves,United States,Active,2017,2017,NaT,50000,5,Totinos,//www.totinos.com/,North America,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both
5,100 Thieves,United States,Active,2017,2017,NaT,50000,5,cashapp,//cash.app/,North America,0,0,0,0,0,0,0,50000,https://www.100thieves.com/,https://twitter.com/100Thieves,NaN,https://www.instagram.com/100thieves,https://facebook.com/100Thieves,https://www.youtube.com/100thieves,nan,NaN,nan,0.000,1.000,0.000,1.000,both
6,1337,Sweden,Inactive,NaT,NaT,2015,6650,0,NaN,NaN,Europe,0,0,150,6500,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,0.000,2.000,0.000,2.000,both
7,2Kill Gaming,Brazil,Inactive,2013,2013,2018,6497,4,Kappa,//www.kappabrasil.com.br,South America,0,0,0,0,3255,3242,0,0,https://www.2killgaming.com,https://twitter.com/2KillGaming,NaN,https://www.instagram.com/2killgaming,https://facebook.com/2killgaming,https://www.youtube.com/channel/2KillGaming,nan,NaN,nan,3.000,3.000,1.000,7.000,both
8,2Kill Gaming,Brazil,Inactive,2013,2013,2018,6497,4,Arsenal XFire,//arsenalxfire.com.br,South America,0,0,0,0,3255,3242,0,0,https://www.2killgaming.com,https://twitter.com/2KillGaming,NaN,https://www.instagram.com/2killgaming,https://facebook.com/2killgaming,https://www.youtube.com/channel/2KillGaming,nan,NaN,nan,3.000,3.000,1.000,7.000,both
9,2Kill Gaming,Brazil,Inactive,2013,2013,2018,6497,4,ASRock,//www.asrock.com/,South America,0,0,0,0,3255,3242,0,0,https://www.2killgaming.com,https://twitter.com/2KillGaming,NaN,https://www.instagram.com/2killgaming,https://facebook.com/2killgaming,https://www.youtube.com/channel/2KillGaming,nan,NaN,nan,3.000,3.000,1.000,7.000,both


In the next part we'll start to analyse this dataset. But for now, as an appetizer, let's find out how the top 10 CS:GO teams has performed in terms of gains, and how much medals and sponsorships they have :

In [40]:
CSGO_Merged.groupby(["team","nbr_sponsors"]).agg({"total_gains":sum,'total_medals':(sum)}).sort_values(["total_gains","total_medals"],ascending=False).head(10)

,,total_gains,total_medals
team,nbr_sponsors,,
Astralis,7,7452634,43.000
Fnatic,4,4934614,173.000
Team Liquid,9,4160812,57.000
SK Gaming,6,3651018,110.000
Natus Vincere,6,3505092,91.000
FaZe Clan,6,3357347,28.000
Virtus.pro,6,3286234,119.000
Ninjas in Pyjamas,7,2830282,104.000
mousesports,5,2448417,108.000


And here are the 10 brands that has sponsored the most CS:GO teams :

In [41]:
CSGO_MI.groupby('brand').agg({'team':(np.size)}).sort_values("team",ascending=False).nlargest(10,"team")

,team
brand,
Twitch,64
HyperX,51
DXRacer,40
Razer,34
SteelSeries,31
Sennheiser,21
AMD,20
MSI,19
Intel,19


## Exporting the two dataframes to csv :

In [42]:
CSGO_Merged.to_csv(os.getcwd() + "\%s.csv" % "CSGO_Merged",index=False)
CSGO_MI.to_csv(os.getcwd() + "\%s.csv" % "CSGO_MI",index=False)